# Формирование запроса к эластик на основе Reka


In [1]:
from pydantic_settings import BaseSettings
import logging
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='../.env')

log_levels = {
    'DEBUG': logging.DEBUG,
    'INFO': logging.INFO,
    'WARNING': logging.WARNING,
    'ERROR': logging.ERROR
}



class Settings(BaseSettings):
    api_name: str = "NLP API"
    logging_level: str = "INFO"

    sentry_dsn: str = ...
    sentry_enable_tracing: bool = True

    elastic_host: str = ...
    elastic_port: int = 9200
    
    project_name: str = ...
    sqlite_path: str = ...
    postgres_version: str = ...
    postgres_ports: str = ...
    postgres_port: str = ...
    postgres_host: str = ...
    postgres_user: str = ...
    postgres_password: str = ...
    postgres_schema_2: str = ...
    postgres_db: str = ...
    postgres_name: str = ...
    elastic_movies_index: str = ...
    stack_version: str = ...
    elastic_cluster_name: str = ...
    memory_lock: str = ...
    elastic_ports: str = ...
    redis_ports: str = ...
    redis_host: str = ...
    redis_port: str = ...
    logstash_ports1: str = ...
    logstash_ports2: str = ...
    logstash_ports3: str = ...
    kibana_ports: str = ...
    es_public_host: str = ...
    logstash_public_host: str = ...
    logstash_host: str = ...
    logstash_port: str = ...
    kibana_public_host: str = ...
    kibana_public_url: str = ...
    elasticsearch_host: str = ...
    es_url: str = ...
    
    yc_api_key: str = ...
    reka_api_key: str = ...

    def get_logging_level(self) -> int:
        return log_levels.get(self.logging_level, logging.INFO)

    class Config:
        env_file = '../.env'


settings = Settings()



In [2]:
import reka
reka.API_KEY = settings.reka_api_key

In [3]:
test = "request is {}"
print (test.format("sasha"))

request is sasha


In [10]:

template = """

You are an assistant that converts natural language queries into Elasticsearch queries.
Convert the following natural language query to an Elasticsearch query:

Query: {}

Please translate any lanquage to english because database language is english. Write please only the Elasticsearch query.

There are 3 indices: movies, genres and persons. Persons could be directors or actors.

movies index schema:

{}

genres index schema:

{}

persons index schema:

{}

"""

In [11]:
import json
from docs.elastic_shemas_config import MOVIES_INDEX_SCHEMA, GENRES_INDEX_SCHEMA, PERSONS_INDEX_SCHEMA

query = "Фильмы джоржа лукоса про войну"
result  = reka.chat(template.format(query, MOVIES_INDEX_SCHEMA, GENRES_INDEX_SCHEMA, PERSONS_INDEX_SCHEMA))
print(json.loads(result['text']))

{'query': {'bool': {'must': [{'match': {'title': {'query': 'Фильмы джоржа лукоса про войну', 'analyzer': 'ru_en'}}}, {'match': {'description': {'query': 'Фильмы джоржа лукоса про войну', 'analyzer': 'ru_en'}}}, {'nested': {'path': 'directors', 'query': {'bool': {'must': [{'match': {'directors.name': {'query': 'джорж лукос', 'analyzer': 'ru_en'}}}]}}, 'score_mode': 'none'}}, {'nested': {'path': 'writers', 'query': {'bool': {'must': [{'match': {'writers.name': {'query': 'джорж лукос', 'analyzer': 'ru_en'}}}]}}, 'score_mode': 'none'}}, {'nested': {'path': 'actors', 'query': {'bool': {'must': [{'match': {'actors.name': {'query': 'джорж лукос', 'analyzer': 'ru_en'}}}]}}, 'score_mode': 'none'}}]}}}


In [12]:
print(template.format(query, MOVIES_INDEX_SCHEMA, GENRES_INDEX_SCHEMA, PERSONS_INDEX_SCHEMA))



You are an assistant that converts natural language queries into Elasticsearch queries.
Convert the following natural language query to an Elasticsearch query:

Query: Фильмы джоржа лукоса про войну

Please translate any lanquage to english because database language is english. Write please only the Elasticsearch query.

There are 3 indices: movies, genres and persons. Persons could be directors or actors.

movies index schema:

{'settings': {'refresh_interval': '1s', 'analysis': {'filter': {'english_stop': {'type': 'stop', 'stopwords': '_english_'}, 'english_stemmer': {'type': 'stemmer', 'language': 'english'}, 'english_possessive_stemmer': {'type': 'stemmer', 'language': 'possessive_english'}, 'russian_stop': {'type': 'stop', 'stopwords': '_russian_'}, 'russian_stemmer': {'type': 'stemmer', 'language': 'russian'}}, 'analyzer': {'ru_en': {'tokenizer': 'standard', 'filter': ['lowercase', 'english_stop', 'english_stemmer', 'english_possessive_stemmer', 'russian_stop', 'russian_stemmer